# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [42]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import AdaBoostClassifier
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import pickle
from sklearn.base import BaseEstimator,TransformerMixin
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO
nltk.download('averaged_perceptron_tagger')
from sklearn import multioutput


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [43]:
# load data from database
engine = create_engine('sqlite:///disasterResponse.db')
df =  pd.read_sql_table("disasterResponse", engine)

In [44]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [46]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
simple_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

simple_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__count_vectorizer', 'features__text_pipeline__tfidf_transformer', 'features__text_pipeline__count_vectorizer__analyzer', 'features__text_pipeline__count_vectorizer__binary', 'features__text_pipeline__count_vectorizer__decode_error', 'features__text_pipeline__count_vectorizer__dtype', 'features__text_pipeline__count_vectorizer__encoding', 'features__text_pipeline__count_vectorizer__input', 'features__text_pipeline__count_vectorizer__lowercase', 'features__text_pipeline__count_vectorizer__max_df', 'features__text_pipeline__count_vectorizer__max_features', 'features__text_pipeline__count_vectorizer__min_df', 'features__text_pipeline__count_vectorizer__ngram_range', 'features__text_pipeline__count_vectorizer__preprocessor', 'features_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [49]:
simple_pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
category = list(df.columns[4:])
Y_pred = simple_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(category)):
    print('Category {}: {} '.format(i, category[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))

Category 0: related 
             precision    recall  f1-score   support

          0       0.58      0.24      0.33      1563
          1       0.79      0.94      0.86      4947
          2       0.40      0.23      0.29        44

avg / total       0.74      0.77      0.73      6554

Accuracy 0.7705218187366494


Category 1: request 
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5404
          1       0.77      0.56      0.65      1150

avg / total       0.89      0.89      0.89      6554

Accuracy 0.8933475740006103


Category 2: offer 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6533
          1       0.14      0.05      0.07        21

avg / total       0.99      1.00      1.00      6554

Accuracy 0.996032956972841


Category 3: aid_related 
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3863
          1       0.

### 6. Improve your model
Use grid search to find better parameters. 

In [51]:
parameters = {
        'vect__max_df':[0.75,1.0],
        'clf__estimator__n_estimators': [20, 50]
    }

cv = GridSearchCV(simple_pipeline, 
                  param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [52]:
cv.fit(X_train, Y_train)
print('Best Parameters:', cv.best_params_)

Best Parameters: {'classifier__estimator__learning_rate': 0.05, 'classifier__estimator__n_estimators': 40}


In [53]:
category_names = list(df.columns[4:])
Y_pred = cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = category_names))

                        precision    recall  f1-score   support

               related       0.89      0.25      0.39      1150
               request       0.00      0.00      0.00        21
                 offer       0.82      0.28      0.42      2691
           aid_related       1.00      0.01      0.02       517
          medical_help       1.00      0.01      0.03       340
      medical_products       1.00      0.02      0.03       191
     search_and_rescue       0.00      0.00      0.00       116
              security       0.00      0.00      0.00       203
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.40      0.54       427
                 water       0.80      0.70      0.75       735
                  food       0.83      0.29      0.43       559
               shelter       0.00      0.00      0.00        90
              clothing       0.00      0.00      0.00       138
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [54]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.